<a href="https://colab.research.google.com/github/Jair2472/Modelos_de_Prediccion/blob/main/Modelo_de_TreeDesicion_%26_RandomForest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Modelo de Predicción MG%**

In [ ]:
# Importar bibliotecas necesarias
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt
import joblib
import os
from google.colab import files

# Cargar el archivo CSV en Colab
print("Por favor, sube el archivo 'BD-Copy1.csv'.")
uploaded = files.upload()
file_path = list(uploaded.keys())[0]  # Obtener el nombre del archivo subido

# Verificar y cargar el archivo
if os.path.exists(file_path):
    try:
        df = pd.read_csv(file_path)
        print("Primeras filas del DataFrame:")
        print(df.head())
        print(f"DataFrame shape: {df.shape}")
    except pd.errors.ParserError:
        print("Error: No se pudo parsear el archivo CSV. Verifica el formato.")
        exit()
    except Exception as e:
        print(f"Error inesperado: {e}")
        exit()
else:
    print("Error: El archivo no se cargó correctamente.")
    exit()

# Convertir la columna 'Date' a datetime
df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')
print("Tipo de dato de 'Date':", df['Date'].dtype)

# Preprocesar los datos
df['Day'] = (pd.to_datetime(df['Date'], format='%d/%m/%Y') - pd.to_datetime(df['Date'].min(), format='%d/%m/%Y')).dt.days + 1
df = df.drop(['FiscalYear', 'Period', 'Depto', 'Date'], axis=1, errors='ignore')

# Computar el margen teórico, evitando división por cero
df['TheoreticalMargenFinal%'] = ((df['VentasAcum'] + df['IngresosAcum'] - df['CostosAcum']) / df['VentasAcum']) * 100
df.loc[df['VentasAcum'] == 0, 'TheoreticalMargenFinal%'] = np.nan

# Feature engineering
df['FormulaIntermediate'] = (df['VentasAcum'] + df['IngresosAcum'] - df['CostosAcum']) / df['VentasAcum']
df.loc[df['VentasAcum'] == 0, 'FormulaIntermediate'] = np.nan

# NO eliminar outliers para preservar todas las filas
# En lugar de eliminar, el pipeline imputará valores faltantes en las features

# Definir features (X) y target (y)
X = df[['Day', 'VentasAcum', 'CostosAcum', 'IngresosAcum', 'FormulaIntermediate']]
y = df['TheoreticalMargenFinal%']

# Crear pipeline con preprocesamiento y modelo
numeric_features = ['Day', 'VentasAcum', 'CostosAcum', 'IngresosAcum', 'FormulaIntermediate']
preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline([
            ('imputer', SimpleImputer(strategy='mean')),  # Imputar valores faltantes
            ('scaler', StandardScaler())
        ]), numeric_features)
    ])

# Pipeline para DecisionTreeRegressor
dt_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', DecisionTreeRegressor(random_state=42))
])

# Optimización de hiperparámetros con GridSearchCV
# Usar solo las filas con y no-NaN para entrenar
valid_indices = ~np.isnan(y)
X_train_valid = X[valid_indices]
y_train_valid = y[valid_indices]

if len(X_train_valid) > 0:  # Asegurarse de que haya datos válidos para entrenar
    param_grid = {
        'regressor__max_depth': [None, 10, 20],
        'regressor__min_samples_split': [2, 5, 10],
        'regressor__min_samples_leaf': [1, 2, 4]
    }
    grid_search = GridSearchCV(dt_pipeline, param_grid, cv=5, scoring='r2', n_jobs=-1)
    X_train, X_test, y_train, y_test = train_test_split(X_train_valid, y_train_valid, test_size=0.2, random_state=42)
    grid_search.fit(X_train, y_train)
    dt_pipeline = grid_search.best_estimator_
    print(f"Mejores parámetros DecisionTree: {grid_search.best_params_}")

    # Evaluar DecisionTree con validación cruzada
    dt_scores = cross_val_score(dt_pipeline, X_train_valid, y_train_valid, cv=5, scoring='r2')
    print(f"DecisionTree Cross-validated R²: {dt_scores.mean():.4f} ± {dt_scores.std():.4f}")

    # Hacer predicciones con DecisionTree
    y_pred_dt = dt_pipeline.predict(X_test)
    dt_mse = mean_squared_error(y_test, y_pred_dt)
    dt_r2 = r2_score(y_test, y_pred_dt)
    print(f"DecisionTree MSE: {dt_mse:.4f}")
    print(f"DecisionTree R²: {dt_r2:.4f}")

    # Probar RandomForest como modelo alternativo
    rf_pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', RandomForestRegressor(random_state=42, n_estimators=100))
    ])
    rf_pipeline.fit(X_train, y_train)
    y_pred_rf = rf_pipeline.predict(X_test)
    rf_mse = mean_squared_error(y_test, y_pred_rf)
    rf_r2 = r2_score(y_test, y_pred_rf)
    print(f"RandomForest MSE: {rf_mse:.4f}")
    print(f"RandomForest R²: {rf_r2:.4f}")

    # Seleccionar el mejor modelo
    best_pipeline = rf_pipeline if rf_r2 > dt_r2 else dt_pipeline
    print(f"Modelo seleccionado: {'RandomForest' if rf_r2 > dt_r2 else 'DecisionTree'}")

    # Generar predicciones para todas las filas del DataFrame original
    df['Predicted_MargenFinal%'] = np.nan  # Inicializar columna
    df.loc[X.index, 'Predicted_MargenFinal%'] = best_pipeline.predict(X)  # Predicciones para todas las filas
else:
    print("Error: No hay datos válidos para entrenar el modelo (todos los valores de y son NaN).")
    exit()

# Visualización de predicciones (solo para datos con y no-NaN)
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred_dt, alpha=0.5, label='DecisionTree')
plt.scatter(y_test, y_pred_rf, alpha=0.5, label='RandomForest')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
plt.xlabel('Valores reales')
plt.ylabel('Predicciones')
plt.title('Predicciones vs Reales')
plt.legend()
plt.show()

# Análisis de residuos
residuals_dt = y_test - y_pred_dt
residuals_rf = y_test - y_pred_rf
plt.figure(figsize=(10, 6))
plt.scatter(y_pred_dt, residuals_dt, alpha=0.5, label='DecisionTree')
plt.scatter(y_pred_rf, residuals_rf, alpha=0.5, label='RandomForest')
plt.axhline(0, color='r', linestyle='--')
plt.xlabel('Predicciones')
plt.ylabel('Residuos')
plt.title('Análisis de Residuos')
plt.legend()
plt.show()

# Predicciones para días futuros (Día 10 al 15)
last_day = df['Day'].max()
last_ventas = df['VentasAcum'].iloc[-1] if not df['VentasAcum'].isna().all() else 0
last_costos = df['CostosAcum'].iloc[-1] if not df['CostosAcum'].isna().all() else 0
last_ingresos = df['IngresosAcum'].iloc[-1] if not df['IngresosAcum'].isna().all() else 0

# Aproximar incrementos diarios
ventas_slope = (last_ventas - df['VentasAcum'].iloc[0]) / (last_day - 1) if last_day > 1 and not np.isnan(last_ventas) else 0
costos_slope = (last_costos - df['CostosAcum'].iloc[0]) / (last_day - 1) if last_day > 1 and not np.isnan(last_costos) else 0
ingresos_slope = (last_ingresos - df['IngresosAcum'].iloc[0]) / (last_day - 1) if last_day > 1 and not np.isnan(last_ingresos) else 0

# Generar datos para días futuros
future_days = range(int(last_day) + 1, int(last_day) + 6)
future_data = []
for day in future_days:
    ventas = last_ventas + ventas_slope * (day - last_day)
    costos = last_costos + costos_slope * (day - last_day)
    ingresos = last_ingresos + ingresos_slope * (day - last_day)
    formula_intermediate = (ventas + ingresos - costos) / ventas if ventas != 0 else np.nan
    future_data.append([day, ventas, costos, ingresos, formula_intermediate])

future_df = pd.DataFrame(future_data, columns=['Day', 'VentasAcum', 'CostosAcum', 'IngresosAcum', 'FormulaIntermediate'])
future_predictions = best_pipeline.predict(future_df)

# Crear DataFrame para días futuros con columnas relevantes
future_df_full = pd.DataFrame({
    'Day': future_days,
    'VentasAcum': future_df['VentasAcum'],
    'CostosAcum': future_df['CostosAcum'],
    'IngresosAcum': future_df['IngresosAcum'],
    'FormulaIntermediate': future_df['FormulaIntermediate'],
    'Predicted_MargenFinal%': future_predictions
})

# Combinar el DataFrame original con los datos futuros
df_combined = pd.concat([df, future_df_full], ignore_index=True, sort=False)

# Mostrar el DataFrame combinado (primeras y últimas filas)
print("\nPrimeras filas del DataFrame con predicciones:")
print(df_combined[['Day', 'VentasAcum', 'CostosAcum', 'IngresosAcum', 'TheoreticalMargenFinal%', 'Predicted_MargenFinal%']].head())
print("\nÚltimas filas del DataFrame con predicciones (incluyendo días futuros):")
print(df_combined[['Day', 'VentasAcum', 'CostosAcum', 'IngresosAcum', 'TheoreticalMargenFinal%', 'Predicted_MargenFinal%']].tail(10))

# Exportar el DataFrame combinado a CSV y descargarlo
df_combined.to_csv('predictions_with_margen.csv', index=False)
files.download('predictions_with_margen.csv')

# Guardar el modelo
joblib.dump(best_pipeline, 'modelo_margen.pkl')
print("Modelo guardado como 'modelo_margen.pkl'")
files.download('modelo_margen.pkl')

Por favor, sube el archivo 'BD-Copy1.csv'.


x